# 빈도 및 교차 데이터 만들기

In [2]:
import pandas as pd # 필요한 라이브러리들
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np

In [3]:
tips = sns.load_dataset("tips")
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [4]:
tips[["sex"]]


,sex
0,Female
1,Male
2,Male
3,Male
4,Female
...,...
239,Male
240,Female
241,Male
242,Male


## <유형별 빈도분석>

In [5]:
tips["sex"].value_counts() # 성별컬럼의 값들에 대한 count 

Male      157
Female     87
Name: sex, dtype: int64

In [6]:
tips["day"].value_counts() #요일별

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [7]:
tips["smoker"].value_counts() #흡연 유무별

No     151
Yes     93
Name: smoker, dtype: int64

In [8]:
tips["time"].value_counts() #시간별

Dinner    176
Lunch      68
Name: time, dtype: int64

In [9]:
# 월별/성별 별 교차분석
pd.crosstab(tips["sex"],tips["day"]) # crosstab이라는 라이브러리에 넣어준다 ->변수는 두개

day,Thur,Fri,Sat,Sun
sex,,,,
Male,30,10,59,58
Female,32,9,28,18


In [15]:
# 성별/ 월별 교차분석 총괄표
pd.crosstab(tips["sex"],
            tips["day"], 
            margins=True)                                                  

day,Thur,Fri,Sat,Sun,All
sex,,,,,
Male,30,10,59,58,157
Female,32,9,28,18,87
All,62,19,87,76,244


In [11]:
# 빈도 비율 확인
pd.crosstab(tips["sex"],
            tips["day"]).apply(
                lambda r: r/len(tips),
                        axis = 1,
                
            )

day,Thur,Fri,Sat,Sun
sex,,,,
Male,0.122951,0.040984,0.241803,0.237705
Female,0.131148,0.036885,0.114754,0.073770


In [12]:
tips.dtypes # 컬럼별 타입

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object

In [13]:
tips.describe() #R에서 summary
#std 중앙값

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


## 데이터 클린징
 - 데이터 전처리에서 가장 중요한 작업중 하나

### 데이터 클린징의 이해
### 1. 결측값
 - 데이터클린징 작업으로 데이터 누락값 즉 결측값이 있는지 검토
 - 결측값이 있는 상태로 분석하는 경우 변수간의 관계가 왜곡될 가능성으로 정확성이 떨어짐
 - 결측값이 발생하는 유형은 다양하고, 무작위로 발생하느냐, 다른변수와 관계가 있는지 여부에 따라 처리하는 방법도 달라짐
 - 판다스에서는 결측값을 NaN으로 표기, None도 결측값을 의미
 - 결측값을 처리하는 방법
  1. 제거와 대체
  2. 제거: 결측값을 포함한 행, 열을 삭제하는 것
  3. 대체: 특정한 방법을 예를 들어 대표값인 평균으로 값을 변환하는 방법

### 1.1 결측 데이터 확인
 - isnull(): 결측 데이터이면 True 값을 반환, 유효한 데이터가 존재하면 False를 반환
 - notnull(): 유효한 데이터가 존재하면 True, 누락데이터면 False를 반환(주로 씀)
 - 결측치 확인시
  1. 보통 결측치가 있는것은 확인 
  2. 그 중 대체할것이 있는지 확인
  3. 대체할 수 있으면 복사본을 생성 평균이나 빈도분석을 통해 대체
  4. notnull()을 이용 대체안한 것들을 걸러냄
  
 ### 1.2 결측치 개수 확인
  - 컬럼별 결측치가 있는 행의 개수 구하기: df.isnull().sum(0)
  - 컬럼별 실제값이 있는 행의 개수 구하기: df.notnull().sum(0)
  - 행 단위로 결측치가 있는 컬럼 개수 구하기: df.isnull().sum(1)
  - 행 단위로 실제값이 있는 컬럼 개수 구하기: df.notnull().sum(1) 단 0, 1은 axis 값이다.
 
 ### 1.3 결측치 제거
  - 행 삭제: 데이터프레임.dropna(axis=0)
  - 열 삭제: 데이터프레임.dropna(axis=1)
  - 데이터프레임.dropna() -> 행과 열의 전체에 하나라도 있으면 해당 행을 삭제, 디폴트값 axis = 0
 
 ### 1.4 결측치 데이터 대체
  - 결측값을 특정 값으로 채우기: df.fillna(0)
  - 결측값을 특정 문자열로 채우기: df.fillna(' ')
  - 결측값을 변수별 평균으로 대체: df.fillna(df.mean())
  
 ### 1.5 결측 데이터 유의사항
  - Null의 의미: 숫자 0과 null과 같은 결측치는 완전히 다른 개념이니 유의해야 함
  - 자료형(실수형, 정수형, 날짜/시간)
  - Null 연산시 유의사항
  - 결측치 제거 전 데이터 백업